In [1]:
%pip install pandas psycopg2-binary python-dotenv
import pandas as pd
import os
from io import StringIO
import psycopg2
from psycopg2.extras import execute_values
import time
from dotenv import load_dotenv

load_dotenv()
db_password = os.environ.get('POSTGRES_PASSWORD')

Note: you may need to restart the kernel to use updated packages.


In [2]:
def create_conn():
    return psycopg2.connect(
        host="localhost",
        port=5432,
        dbname="mydb",
        user="danny",
        password = db_password,
    )

In [3]:
product_file = "./db/data/product.csv"
df_products = pd.read_csv(product_file)

features_file = "./db/data/features.csv"
df_features = pd.read_csv(features_file)

skus_file = "./db/data/skus.csv"
df_skus = pd.read_csv(skus_file)

styles_file = "./db/data/styles.csv"
df_styles = pd.read_csv(styles_file)

related_file = "./db/data/related.csv"
df_related = pd.read_csv(related_file)

photos_file = "./db/data/photos.csv"
columns_to_use = [0, 1, 2, 3] # Only use the first 4 columns
df_photos = pd.read_csv(photos_file, usecols=columns_to_use)

In [5]:
def extract_from_csv(df, table_name):
    conn = create_conn()
    cur = conn.cursor()

    # drop_table_sql = f"DROP TABLE IF EXISTS {table_name};" # Drop the table if it exists
    drop_table_sql = f"DROP TABLE IF EXISTS {table_name} CASCADE;" # Drop table and all its dependencies
    cur.execute(drop_table_sql)

    # Dynamically generate the CREATE TABLE statement based on headers and first row data
    headers = df.columns.tolist()
    first_row = df.iloc[0]
    columns = []
    primary_key = ""
    foreign_key = ""
    
    # Replace "productId" and "current_product_id" with "product_id" for uniformity
    headers = ["product_id" if header in ["productId", "current_product_id"] else header for header in headers]
    headers = ["style_id" if header in ["styleId"] else header for header in headers]

    for header, value in zip(headers, first_row):
        data_type = "VARCHAR" if isinstance(value, str) else "INTEGER"
        columns.append(f"{header} {data_type}")
        
        # Define primary & foreign keys
        if header == "id":
            primary_key = "PRIMARY KEY (id)"
            
        if header == "product_id":
            foreign_key = f"FOREIGN KEY (product_id) REFERENCES products (id)"
        elif header == "style_id":
            foreign_key = f"FOREIGN KEY (style_id) REFERENCES styles (id)"
        elif header == "related_product_id" and table_name == "related_items":
            foreign_key = f"FOREIGN KEY (related_product_id) REFERENCES products (id)"
    
    create_table_sql = f"CREATE TABLE {table_name} ({', '.join(columns)}" + (f", {primary_key}" if primary_key else "") + (f", {foreign_key}" if foreign_key else "") + ");"
    cur.execute(create_table_sql)

    # Create indexes for the keys
    index_columns = ["id", "style_id", "product_id", "related_product_id"]
    for col in index_columns:
        if col in headers:
            cur.execute(f"CREATE INDEX {table_name}_{col}_idx ON {table_name} ({col});")

    conn.commit()
    df.columns = headers #update headers
    
    if table_name == "styles":
        df['sale_price'] = df['sale_price'].apply(lambda x: 0 if pd.isna(x) or x == 'null' else int(round(float(x), 2))).astype('Int64')


    insert_data(df, table_name, cur)
    conn.commit()

    cur.execute(f"SELECT * FROM {table_name} LIMIT 5")

    results = cur.fetchall()

    for row in results:
        print(row)

    cur.close()
    conn.close()


def insert_data(df, table_name, cur):
    # Filter out the rows with non-existent related_product_id's
    if table_name == "related_items":
        conn_subquery = create_conn()
        df = df.loc[df["related_product_id"].isin(pd.read_sql("SELECT id FROM products", conn_subquery).id.tolist())]
        conn_subquery.close()

    # Prepare the data as a CSV string
    csv_buffer = StringIO()
    df.to_csv(csv_buffer, index=False, header=False)
    csv_buffer.seek(0)

    # import data
    columns = ", ".join(df.columns)
    cur.copy_expert(f"COPY {table_name} ({columns}) FROM STDIN WITH CSV", csv_buffer)
    

In [6]:
# Record the start and end time, then calculate duration
start_time = time.time()
extract_from_csv(df_products, "products")
extract_from_csv(df_related, "related_items")
extract_from_csv(df_styles, "styles")
extract_from_csv(df_features, "features")
extract_from_csv(df_photos, "photos")
extract_from_csv(df_skus, "skus")

end_time = time.time()

duration = end_time - start_time
print(f"Data import took {duration:.2f} seconds.")

(1, 'Camo Onesie', 'Blend in to your crowd', 'The So Fatigues will wake you up and fit you in. This high energy camo will have you blending in to even the wildest surroundings.', 'Jackets', 140)
(2, 'Bright Future Sunglasses', "You've got to wear shades", "Where you're going you might not need roads, but you definitely need some shades. Give those baby blues a rest and let the future shine bright on these timeless lenses.", 'Accessories', 69)
(3, 'Morning Joggers', 'Make yourself a morning person', "Whether you're a morning person or not.  Whether you're gym bound or not.  Everyone looks good in joggers.", 'Pants', 40)
(4, "Slacker's Slacks", 'Comfortable for everything, or nothing', "I'll tell you how great they are after I nap for a bit.", 'Pants', 65)
(5, 'Heir Force Ones', 'A sneaker dynasty', "Now where da boxes where I keep mine? You should peep mine, maybe once or twice but never three times. I'm just a sneaker pro, I love Pumas and shell toes, but can't nothin compare to a fres

/tmp/ipykernel_742/396508379.py:69: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = df.loc[df["related_product_id"].isin(pd.read_sql("SELECT id FROM products", conn_subquery).id.tolist())]


(1, 1, 2)
(2, 1, 3)
(3, 1, 8)
(4, 1, 7)
(5, 2, 3)
(1, 1, 'Forest Green & Black', 0, 140, 1)
(2, 1, 'Desert Brown & Tan', 0, 140, 0)
(3, 1, 'Ocean Blue & Grey', 100, 140, 0)
(4, 1, 'Digital Red & Black', 0, 140, 0)
(5, 1, 'Sky Blue & White', 100, 140, 0)
(1, 1, 'Fabric', 'Canvas')
(2, 1, 'Buttons', 'Brass')
(3, 2, 'Lenses', 'Ultrasheen')
(4, 2, 'UV Protection', None)
(5, 2, 'Frames', 'LightCompose')
(1, 1, 'https://images.unsplash.com/photo-1501088430049-71c79fa3283e?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&auto=format&fit=crop&w=668&q=80', 'https://images.unsplash.com/photo-1501088430049-71c79fa3283e?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&auto=format&fit=crop&w=300&q=80')
(2, 1, 'https://images.unsplash.com/photo-1534011546717-407bced4d25c?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&auto=format&fit=crop&w=2734&q=80', 'https://images.unsplash.com/photo-1534011546717-407bced4d25c?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&auto=format&fit=crop&w=300&q=80')
(3, 1, 'https://images.unsplash.com